In [1]:
from bs4 import BeautifulSoup

import requests,re

import pandas as pd

from urllib.parse import urlparse

In [2]:
url_base='https://www.vghtc.gov.tw'

url_work_table='/Module/Recruitment?WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78'
url_admit = '/Module/Admission?WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78'

url=url_base+url_work_table

g=requests.get(url)

soup=BeautifulSoup(g.content, 'html.parser')

In [3]:
soup.find('table').find_all('tr')

[<tr>
 <th><div style="text-align:left;">更新日期</div></th>
 <th>徵才類別</th>
 <th><div style="text-align:left;">徵才項目</div></th>
 <th><div style="text-align:left;">報名截止日</div></th>
 </tr>,
 <tr>
 <td data-th="更新日期">2023/2/8</td>
 <td data-th="徵才類別">行政人員</td>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=5906b762-a5a7-4484-8298-6e1f4b2f36cd&amp;WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78">醫務企管部病醫療事務組 誠徵契約行政助理4名</a></td>
 <td data-th="報名截止日">2023/2/19</td>
 </tr>,
 <tr>
 <td data-th="更新日期">2023/2/8</td>
 <td data-th="徵才類別">技術人員</td>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=45945bd9-b88d-40b0-a242-e6e874895b1f&amp;WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78">【病理檢驗部】誠徵契約醫事檢驗師1名</a></td>
 <td data-th="報名截止日">2023/2/13</td>
 </tr>,
 <tr>
 <td data-th="更新日期">2023/2/8</td>
 <td data-th="徵才類別">技術人員</td>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=96e8b638-6b7a-4bd

In [4]:
for tt in soup.find('table').find_all('tr'):
    for ttt in tt.findAll('td'):
        if ttt.get('data-th') == '更新日期':
            print('data',ttt.text)
        elif ttt.get('data-th') == '徵才類別':
            print('召聘職稱',ttt.text)
        elif ttt.get('data-th') == '報名截止日':
            print('期限' , ttt.text)
        elif ttt.get('data-th') == '徵才項目':
            job_detail_link = ttt.find('a')
            print('詳細連結', url_base+job_detail_link.get('href'))
        else:
            print(ttt)


data 2023/2/8
召聘職稱 行政人員
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=5906b762-a5a7-4484-8298-6e1f4b2f36cd&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2023/2/19
data 2023/2/8
召聘職稱 技術人員
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=45945bd9-b88d-40b0-a242-e6e874895b1f&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2023/2/13
data 2023/2/8
召聘職稱 技術人員
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=96e8b638-6b7a-4bd2-8a1e-aaf7aa964cba&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2023/2/14
data 2023/2/7
召聘職稱 依時計時人員
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=c2deed41-a014-46a7-b419-a9c7e783d6a3&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2023/2/24
data 2023/2/6
召聘職稱 住院醫師
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=31b7ba14-4981-41e9-8927-060a4c8f1021&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2023/2/20
data 2023/2/6
召聘職稱 行政人員
詳細連結 https://www.vg

In [5]:
work_table=[]
for tt in soup.find('table').find_all('tr'):
    _one_job_data ={}
    for ttt in tt.findAll('td'):
        
        if ttt.get('data-th') == '更新日期':
            #print('data',ttt.text)
            pass
        elif ttt.get('data-th') == '報名截止日':
            #print('期限' , ttt.text)
            _one_job_data['deadline'] = ttt.text
        elif ttt.get('data-th') == '徵才項目':
            clean_text = ttt.text.replace('【徵才公告】','').replace("【徵的就是你/妳】","")
            _one_job_data['jobtype'] = clean_text
            try:
                _one_job_data['job_type'] = re.search(r"[放,復,營,資,護,家,胸,皮,泌,眼,內,外,藥,麻,臨,急,病,兒,環,健]\w+[科,部,室,心,課]",clean_text).group(0)
            except:
                print(ttt.text,'error find job_type')
            job_detail_link = url_base+ttt.find('a').get('href')
            #print('詳細連結', job_detail_link)
            _one_job_data['job_detail_link'] = job_detail_link
            g=requests.get(job_detail_link)
            _soup=BeautifulSoup(g.content, 'html.parser')
            try:
                job_file_link = url_base+_soup.find('a',_class="btn-recruit-add").get('href')
            except:
                job_file_link = None
            _one_job_data['job_file_link'] = job_file_link
    print(_one_job_data)
    if _one_job_data:
        work_table.append(_one_job_data)
    print(pd.DataFrame(work_table))

{}
Empty DataFrame
Columns: []
Index: []
醫務企管部病醫療事務組 誠徵契約行政助理4名 error find job_type
{'jobtype': '醫務企管部病醫療事務組 誠徵契約行政助理4名', 'job_detail_link': 'https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=5906b762-a5a7-4484-8298-6e1f4b2f36cd&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78', 'job_file_link': None, 'deadline': '2023/2/19'}
                  jobtype                                    job_detail_link  \
0  醫務企管部病醫療事務組 誠徵契約行政助理4名  https://www.vghtc.gov.tw/Module/RecruitmentCon...   

  job_file_link   deadline  
0          None  2023/2/19  
{'jobtype': '【病理檢驗部】誠徵契約醫事檢驗師1名', 'job_type': '病理檢驗部', 'job_detail_link': 'https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=45945bd9-b88d-40b0-a242-e6e874895b1f&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78', 'job_file_link': None, 'deadline': '2023/2/13'}
                  jobtype                                    job_detail_link  \
0  醫務企管部病醫療事務組 誠徵契約行政助理4名  https://www.vghtc.gov.tw/Module/RecruitmentCon...   
1     

In [6]:
pd.DataFrame(work_table)

,jobtype,job_detail_link,job_file_link,deadline,job_type
0,醫務企管部病醫療事務組 誠徵契約行政助理4名,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/19,NaN
1,【病理檢驗部】誠徵契約醫事檢驗師1名,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/13,病理檢驗部
2,【病理檢驗部】微生物科誠徵定期契約醫事檢驗師(職務代理),https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/14,病理檢驗部
3,麻醉部誠徵工時研究助理,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/24,麻醉部
4,核醫科招訓112年度第一年住院醫師甄選公告,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/20,NaN
5,資訊室徵：【契約資訊副工程師】(後端工程師 Backend Engineer)一名,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/15,資訊室
6,定期契約醫事技術師(物理治療師),https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/10,NaN
7,護理部誠徵儲備心導管室契約護理,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/17,護理部誠徵儲備心導管室
8,藥學部誠徵契約(含定期)藥師,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/12,藥學部
9,婦女醫學部誠徵定期契約醫事技術師一名(超音波室),https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/17,NaN


# Get page link
In this website the work table has some page, so the function should find page link for next step.

In [9]:
pages_link = []

pages_link.append(url)

for pp in soup.find('div', class_="pager").find_all('a'):
    if pp.get('title') == 'Next page':
        pass
    else:
        pages_link.append(url_base+pp.get('href'))
pages_link

['https://www.vghtc.gov.tw/Module/Recruitment?WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78',
 'https://www.vghtc.gov.tw/Module/Recruitment?WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78&page=2']

# Get admit table

The get_work_page function is used for getting each work page


In [7]:
url=url_base+url_admit

g=requests.get(url)

admit_page_base=BeautifulSoup(g.content, 'html.parser')


admit_tables = admit_page_base.find_all('tr')

In [8]:
admit_tables

[<tr>
 <th><div style="text-align:left;">徵才項目</div></th>
 <th><div style="text-align:left;">錄取名單</div></th>
 <th><div style="text-align:left;">公告迄日</div></th>
 </tr>,
 <tr>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=49bd1528-85d5-4608-8cb5-620586e8d2f6&amp;WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78">【徵才公告】臺中榮民總醫院兒童醫學中心誠徵契約醫事技術師1名</a></td>
 <td class="aboutNews" data-th="錄取名單"><a href="/UploadFiles/Files/RecruitAttach/兒童醫學中心契約醫事技術師錄取公告.pdf" target="_blank">正取1名（備取1名)</a></td>
 <td data-th="公告迄日">2023/5/1</td>
 </tr>,
 <tr>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=e80f9124-955e-4a90-a9c4-0e44f327e2df&amp;WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78">【徵才公告】婦女醫學部誠徵契約醫事技術師一名(細胞遺傳檢驗室) (二次公告)</a></td>
 <td class="aboutNews" data-th="錄取名單"><a href="/UploadFiles/Files/RecruitAttach/進用郭穎蓁簽案-奉准.pdf" target="_blank">正取1名/備取2名</a></td>
 <td data-th="公告迄日">2023/4/30</td>
 </tr>,
 <tr>
 <td class="aboutN

In [10]:
admit_table = []
for tt in admit_tables:
    _admit_data = {}
    for ttt in tt.findAll('td'):
        if ttt.get('data-th') == "徵才項目":
            _admit_data['召聘職稱'] = ttt.text.replace("【錄取公告】","")
        elif ttt.get('data-th') == '公告迄日':
            _admit_data['期限'] = ttt.text
        elif ttt.get('data-th') == '錄取名單':
            job_detail_link = ttt.find('a')
            _admit_data['連結'] = url_base+job_detail_link.get('href')
        else:
            print("Error",ttt, "other message")
    if _admit_data != {}:
        admit_table.append(_admit_data)

In [11]:
admit_table

[{'召聘職稱': '【徵才公告】臺中榮民總醫院兒童醫學中心誠徵契約醫事技術師1名',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/兒童醫學中心契約醫事技術師錄取公告.pdf',
  '期限': '2023/5/1'},
 {'召聘職稱': '【徵才公告】婦女醫學部誠徵契約醫事技術師一名(細胞遺傳檢驗室) (二次公告)',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/進用郭穎蓁簽案-奉准.pdf',
  '期限': '2023/4/30'},
 {'召聘職稱': '【徵才公告】胸腔內科徵：契約醫事技術師1名',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/1672274374065.jpg',
  '期限': '2023/4/3'},
 {'召聘職稱': '皮膚科招訓112年度第一年住院醫師',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/臺中榮總皮膚科112年度第一年住院醫師錄取名單.jpg',
  '期限': '2023/3/21'},
 {'召聘職稱': '台中榮總家庭醫學部112年第一年住院醫師',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/臺中榮總家庭醫學部112年第一年住院醫師錄取名單.pdf',
  '期限': '2023/3/7'},
 {'召聘職稱': '藥學部誠徵契約(含定期)藥師',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/',
  '期限': '2023/2/1'},
 {'召聘職稱': '胸腔內科徵：師三級主治醫師1名',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/',
  '期限': '2023/1/3'},
 {'召聘職稱'

In [12]:
pd.DataFrame(admit_table, columns=['召聘職稱','期限' ,'連結'])

,召聘職稱,期限,連結
0,【徵才公告】臺中榮民總醫院兒童醫學中心誠徵契約醫事技術師1名,2023/5/1,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
1,【徵才公告】婦女醫學部誠徵契約醫事技術師一名(細胞遺傳檢驗室) (二次公告),2023/4/30,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
2,【徵才公告】胸腔內科徵：契約醫事技術師1名,2023/4/3,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
3,皮膚科招訓112年度第一年住院醫師,2023/3/21,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
4,台中榮總家庭醫學部112年第一年住院醫師,2023/3/7,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
5,藥學部誠徵契約(含定期)藥師,2023/2/1,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
6,胸腔內科徵：師三級主治醫師1名,2023/1/3,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
7,【徵才公告】醫院整合醫學暨醫療銜接照護試辦計畫研究助理1名,2022/12/28,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
8,藥學部誠徵契約(含定期)藥師,2022/11/30,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
9,藥學部誠徵半日契約藥師1名,2022/10/26,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
